In [2]:
from google.cloud import storage
from google.cloud import bigquery

/home/alee66831/.ccdr/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
from google.cloud import storage
from google.cloud import bigquery

def upload_cc_bq(event):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket("ccdr-raw")
    blobs = bucket.list_blobs()

    file_names = []
    for blob in blobs:
        file_names.append(blob.name)
    for i in range(len(file_names)):
        database = 'ccdr-265306'
        bigquery_client = bigquery.Client(database)
        dataset = bigquery_client.dataset('ccdr_database')
        table = dataset.table('credit_cardholder_data')
        job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")
        job_config.skip_leading_rows = 1
        job = bigquery_client.load_table_from_uri('gs://ccdr-raw/' + file_names[i], 
                                                  table, job_config = job_config)
        job.job_id
        job.result()
        if job.state == 'DONE':
            print("state of job is: " + job.state)
        else: 
            print(job.errors)
    return